# Ejemplo de bioinformática

En este ejemplo vamos a analizar datos de un dataset de bioinformática llamado Gen Expression Omnibus (GEO), disponible en el curso virtual y que originalmente fue descargado del siguiente enlace:

https://www.ncbi.nlm.nih.gov/geo/info/geo_paccess.html#Programs

En concreto, utilizaremos el fichero GSD1001_full.soft.txt que presenta los datos de una serie de experimentos en bioinformática. El contenido del fichero es una serie de filas con las siguientes columnas separadas por tabuladores:

ID_REF - IDENTIFIER - GSM19023 - GSM19024 - GSM19025 - GSM19026 - Gene title - Gene symbol - Gene ID - UniGene title - UniGene symbol - UniGene ID - Nucleotide Title - GI - GenBank Accession - Platform_CLONEID - Platform_ORF - Platform_SPOTID - Chromosome location - Chromosome annotation - GO:Function - GO:Process - GO:Component - GO:Function ID - GO:Process ID - GO:Component ID


### Este ejercicio consiste en calcular la media de la columna GSD19026 pero solamente teniendo en cuenta las filas cuyo campo GenTitle vale angiopoietin-like 4

Para una definición del campo GSM19026, se puede consultar el siguiente enlace: 

https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM19026 

In [1]:
! mkdir -p ejemplo-inicial/bioinformatica

In [2]:
import os
os.chdir("/media/notebooks/ejemplo-inicial/bioinformatica")

In [3]:
! pwd

/media/notebooks/ejemplo-inicial/bioinformatica


# Debemos haber descargado el fichero GDS1001_full.soft.txt y dejado en /media/notebooks/ejemplo-inicial/bioinformatica

In [4]:
! hdfs dfs -rm /tmp/ejemplobioinformatica/*
! hdfs dfs -rmdir /tmp/ejemplobioinformatica

Deleted /tmp/ejemplobioinformatica/GDS1001_full.soft.txt


In [5]:
! hdfs dfs -mkdir /tmp/ejemplobioinformatica

In [6]:
! hdfs dfs -put GDS1001_full.soft.txt /tmp/ejemplobioinformatica

In [7]:
%%writefile mapper.py
#!/usr/bin/python
 
import sys
 
for line in sys.stdin:
  data = line.strip().split("\t")

  if len(data) == 26:
    idref, ident, gsm19023, gsd19024, gsd19025, \
    gsd19026, genetitle, genesymbol, geneID, uniGenetitle, \
    uniGenesymbol, uniGeneID, NucleotideTitle, GI, \
    GenBankAccession, PlatformCLONEID, PlatformORF, PlatformSPOTID, \
    Chromosomelocation, Chromosomeannotation, GOFunction,GOProcess, \
    GOComponent, GOFunctionID,  GOProcessID, GOComponentID = data

    if genetitle == "angiopoietin-like 4":
        print "{0}\t{1}".format(genetitle, gsd19026)

Overwriting mapper.py


In [8]:
%%writefile reducer.py
#!/usr/bin/python
 
import sys
 
GSDTotal = 0
oldKey = None
GSDcounter = 0
GSDavg = 0
 
# Recorremos los datos de entrada, que estan en formato clave/valor
# donde la clave es el genetitle y valor es el de la columna gsd19026
 
for line in sys.stdin:
  data_mapped = line.strip().split("\t")
  if len(data_mapped) != 2:
    # Algo ha pasado, nos saltamos esta linea
    continue
 
  thisKey, thisGSD = data_mapped
 
  GSDTotal += float(thisGSD)
  GSDcounter += 1
  print thisKey, "\t", thisGSD

    
GSDavg = GSDTotal/GSDcounter
print "\nLa media de GSD19026 para las muestras cuyo GenTitle vale angiopoietin-like 4 es ... \t", GSDavg

Overwriting reducer.py


In [9]:
! hdfs dfs -rm  /tmp/salida-ejemplobioinformatica/*
! hdfs dfs -rmdir /tmp/salida-ejemplobioinformatica

rm: `/tmp/salida-ejemplobioinformatica/*': No such file or directory
rmdir: `/tmp/salida-ejemplobioinformatica': No such file or directory


In [10]:
! hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /tmp/ejemplobioinformatica/* \
-output /tmp/salida-ejemplobioinformatica

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob5596744979984745312.jar tmpDir=null
19/08/12 06:55:42 INFO client.RMProxy: Connecting to ResourceManager at yarnmaster/172.21.0.3:8032
19/08/12 06:55:43 INFO client.RMProxy: Connecting to ResourceManager at yarnmaster/172.21.0.3:8032
19/08/12 06:55:43 INFO mapred.FileInputFormat: Total input paths to process : 1
19/08/12 06:55:43 INFO mapreduce.JobSubmitter: number of splits:2
19/08/12 06:55:43 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1565590814059_0003
19/08/12 06:55:43 INFO impl.YarnClientImpl: Submitted application application_1565590814059_0003
19/08/12 06:55:43 INFO mapreduce.Job: The url to track the job: http://yarnmaster:8088/proxy/application_1565590814059_0003/
19/08/12 06:55:43 INFO mapreduce.Job: Running job: job_1565590814059_0003
19/08/12 06:55:49 INFO mapreduce.Job: Job job_1565590814059_0003 running in uber mode : false
19/08/12 06:55:49 INFO mapreduce.Job

In [11]:
! hdfs dfs -cat /tmp/salida-ejemplobioinformatica/part-00000

angiopoietin-like 4 	124.4
angiopoietin-like 4 	634.3
angiopoietin-like 4 	767.2
angiopoietin-like 4 	125.5
	
La media de GSD19026 para las muestras cuyo GenTitle vale angiopoietin-like 4 es ... 	412.85
